# A RAG system for the Kenyan Constitution

In [1]:
import pdfplumber
# import spacy
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline
import chromadb
from chromadb.utils import embedding_functions
import os
from dotenv import load_dotenv

load_dotenv()

c:\Users\admin\Documents\LuxDev\Law-RAG-System\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'chromadb'